In [3]:
import numpy as np
import scipy
import scipy.io
import csv
import sklearn
import sklearn.feature_extraction

spam = scipy.io.loadmat("./dist/spam_data.mat")
census_train = csv.DictReader(open("./census_training_data.csv"))
census_test = csv.DictReader(open("./census_test_data.csv"))
titanic_train = csv.DictReader(open("./titanic_training_data.csv"))
titanic_test = csv.DictReader(open("./titanic_testing_data.csv"))

firstTest = None
for row in census_train:
    print(row)
    firstTest = row
    break

{'native-country': 'United-States', 'fnlwgt': '307423', 'age': '59', 'relationship': 'Not-in-family', 'race': 'Black', 'label': '0', 'education-num': '5', 'workclass': 'Private', 'capital-loss': '0', 'capital-gain': '0', 'marital-status': 'Never-married', 'sex': 'Male', 'hours-per-week': '50', 'education': '9th', 'occupation': 'Other-service'}


In [4]:

vec = sklearn.feature_extraction.DictVectorizer()
X = vec.fit_transform(census_train)


ValueError: Sample sequence X is empty.

In [20]:
class Node:
    def __init__(self):
        self.split_rule = None
        self.left = None
        self.right = None
        self.label = None 

def nodeDecision(node, data):
    if node.split_rule != None:
        if data[node.split_rule[0]] == node.split_rule[1]:
            return nodeDecision(node.right, data)
        else:
            return nodeDecision(node.left, data)
    else:
        return node.label

In [11]:
# Where left and right are maps from labels -> frequencies
def impurity(left, right):
    leftEn = 0
    rightEn = 0
    leftSum = 0
    rightSum = 0
    for k,v in left.items():
        p_c = v / len(left)
        if v == 0: continue
        leftEn += p_c * np.log2(p_c)
        leftSum += v
    for k,v in right.items():
        p_c = v / len(right)
        if v == 0: continue
        rightEn += p_c * np.log2(p_c)
        rightSum += v
    weightSum = leftSum * leftEn + rightSum * rightEn
    return weightSum / (leftEn + rightEn)

In [26]:
# Return None if the data is essentially pure across 
def segmenter(data, labels):
    uniqueLabels = set(labels)
    minBadness = -1
    maxLabelIndex = -1
    maxLabelSplitDir = -1
    
    #TODO: Confirm type of data and figure out type of labels
    
    if len(labels) == 0:
        return None
    
    firstLabel = labels[0]
    pure = True
    for label in labels:
        if firstLabel != label:
            pure = False
            break
    if pure:
        return None
    
    for featureIndex in range(len(data[0])):
        splits = [0] #With binary variables, a 0 split 
                        #is equivalent to a 1 split 
        for split in splits:
            left = dict()
            right = dict()
            for uniqueLabel in uniqueLabels:
                left[uniqueLabel] = 0
                right[uniqueLabel] = 0
            
            """
            for dataIndex in range(len(data)):
                if data[dataIndex][labelIndex] != split:
                    for labelIndex2 in range(len(labels)):
                        left[labelIndex2] += 1
                else:
                    for labelIndex2 in range(len(labels)):
                        right[labelIndex2] += 1
            """
            
            for dataIndex in range(len(data)):
                if data[dataIndex][featureIndex] != split:
                    left[labels[dataIndex]] += 1
                else:
                    right[labels[dataIndex]] += 1
                    
            badness = impurity(left, right)
            
            #print(str(labelIndex) + " " + str(split) + " " + str(badness))
            #print(str(left["a"]) + " " + str(left["b"]))
            #print(str(right["a"]) + " " + str(right["b"]))
            #print("----------")
            
            if badness < minBadness or maxLabelIndex == -1:
                minBadness = badness
                maxLabelIndex = featureIndex
                maxLabelSplitDir = split
    return (maxLabelIndex, maxLabelSplitDir)

test = np.array([
        [1,1,0,1],
        [1,1,0,0],
        [1,1,1,1],
        [1,1,1,1],
        [1,1,0,1],
        [1,1,1,1],
        [1,1,0,1],
        [1,1,0,0],
        [1,1,1,1],
        [1,1,1,1],
        [1,1,0,1],
        [1,1,1,1],
        [1,1,1,1],
        [1,1,1,1]
    ])
segmenter(test, ["a","a","b","a","b","a","b","b","b","a","a","b","a","a"])

test = np.array([
        [1,1,0,1],
        [1,0,1,1],
        [0,1,1,1],
        [1,1,1,0],
        [1,1,0,1],
        [1,1,0,1],
        [1,1,0,1],
        [1,1,0,1]
    ])
segmenter(test, ["a","a","a","a","b","b","b","b"])

test = np.array([
        [1,1,1,1],
        [1,1,1,1],
        [1,1,1,1],
        [1,1,1,1],
        [1,1,0,1],
        [1,1,0,1],
        [1,1,0,1],
        [1,1,0,1]
    ])
segmenter(test, ["a","a","a","a","b","b","b","b"])

(2, 0)

In [32]:
initialNode = Node()

def train(node, data, labels, levels=3):
    if levels <= 0: return
    decision = segmenter(data, labels)
    print("---------------")
    print("For data set: ")
    print(data)
    print("and labels: ")
    print(labels)
    if decision != None: #A split rule was found for a non-pure node
        print("A decision rule was found: ")
        print(decision)
        node.split_rule = decision
        node.left = Node()
        node.right = Node()
        leftData = []
        rightData = []
        leftLabels = []
        rightLabels = []
        for dataIndex in range(len(data)):
            if data[dataIndex][decision[0]] == decision[1]:
                rightData.append(data[dataIndex])
                rightLabels.append(labels[dataIndex])
            else:
                leftData.append(data[dataIndex])
                leftLabels.append(labels[dataIndex])
        print(str(len(leftData)) + " " + str(len(rightData)))
        leftData = np.array(leftData)
        rightData = np.array(rightData)
        train(node.left, leftData, leftLabels, levels - 1)
        train(node.right, rightData, rightLabels, levels - 1)
    else:
        print("No decision was found for a pure node")
        return
    
testData = np.array([
        [1,1,0,1],
        [1,1,0,0],
        [1,1,1,1],
        [1,1,1,1],
        [1,1,0,1],
        [1,1,1,1],
        [1,1,0,1],
        [1,1,0,0],
        [1,1,1,1],
        [1,1,1,1],
        [1,1,0,1],
        [1,1,1,1],
        [1,1,1,1],
        [1,1,1,1]
    ])
testLabels = np.array(
    ["a","a","b","a","b","a","b","b","b","a","a","b","a","a"]
)

train(initialNode, testData, testLabels, 3)

---------------
For data set: 
[[1 1 0 1]
 [1 1 0 0]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 0 1]
 [1 1 1 1]
 [1 1 0 1]
 [1 1 0 0]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 0 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
and labels: 
['a' 'a' 'b' 'a' 'b' 'a' 'b' 'b' 'b' 'a' 'a' 'b' 'a' 'a']
A decision rule was found: 
(2, 0)
8 6
---------------
For data set: 
[[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
and labels: 
['b', 'a', 'a', 'b', 'a', 'b', 'a', 'a']
A decision rule was found: 
(0, 0)
8 0
---------------
For data set: 
[[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
and labels: 
['b', 'a', 'a', 'b', 'a', 'b', 'a', 'a']
A decision rule was found: 
(0, 0)
8 0
---------------
For data set: 
[]
and labels: 
[]
No decision was found for a pure node
---------------
For data set: 
[[1 1 0 1]
 [1 1 0 0]
 [1 1 0 1]
 [1 1 0 1]
 [1 1 0 0]
 [1 1 0 1]]
and labels: 
['a', 'a', 'b', 'b', 'b', 'a']
A decision rule was found: 
(3, 0)
4 2
---------

c:\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: invalid value encountered in double_scalars
